# Multi-user ReAct Agent Chatbot in LangChain
* Notebook by Adam Lang
* Date: 10/2/2024

# Overview
* In this notebook we will build a multi-user ReAct AI Agent chatbot using LangChain Legacy Syntax.
* Tools we will use:
1. web search -- Tavily API
2. weather API
* To make this "Multi-user" we will store the converation in memory.

## Install dependencies

In [1]:
!pip install langchain==0.2.0
!pip install langchain-openai==0.1.7
!pip install langchain-community==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.0/397.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383

## Enter Open AI API Key

In [2]:
from getpass import getpass

OPENAI_KEY = getpass("Enter your OPEN AI API KEY: ")

Enter your OPEN AI API KEY: ··········


## Enter Tavily Search API Key

In [3]:
TAVILY_API_KEY = getpass("Enter your TAVILY API KEY: ")

Enter your TAVILY API KEY: ··········


## Enter WeatherAPI Key

In [4]:
WEATHER_API_KEY = getpass("Enter your WeatherAPI KEY: ")

Enter your WeatherAPI KEY: ··········


## Setup Environment Variables

In [5]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

# Create Custom Tools
* We are going to create 2 custom tools which will serve as wrappers on top of the **TAVILY API** and **WEATHER API** respectively.
* The tools are:
  * 1) Simple Web Search Tool
  * 2) Weather tool

In [6]:
## imports
from langchain_community.tools.tavily_search import TavilySearchResults ## Tavily search
from langchain_core.tools import tool ## tool
import requests
import json

## instantiate tavily search
tv_search = TavilySearchResults(max_results=3,
                                search_depth='advanced',
                                max_tokens=10000)

## search tool
@tool
def search_web(query: str) -> list:
  """Custom tool to search web for a query."""
  tavily_tool = TavilySearchResults(max_results=2) ## returns list of search results
  results = tavily_tool.invoke(query) ## invoke tool with query
  return results


## weather tool
@tool
def get_weather(query: str) -> list:
  """Custom tool to search WeatherAPI for current weather."""
  base_url = "http://api.weatherapi.com/v1/current.json"
  complete_url = f"{base_url}?key={WEATHER_API_KEY}&q={query}"

  ## get API response
  response = requests.get(complete_url)
  data = response.json()
  if data.get("location"):
    return data
  else:
    return "Weather Data Not Found!"

# Test Tool Calling with LLM

In [7]:
from langchain_openai import ChatOpenAI

##setup LLM
llm = ChatOpenAI(model='gpt-4o', temperature=0)
tools = [search_web, get_weather]

## instantiate tools
llm_with_tools = llm.bind_tools(tools)

In [8]:
## prompt test
prompt = "Who won the Stanley Cup in 2011"
response = llm_with_tools.invoke(prompt)
response.tool_calls

[]

In [9]:
## prompt test
prompt = "how is the weather in Denver today?"
response = llm_with_tools.invoke(prompt)
response.tool_calls

[{'name': 'get_weather',
  'args': {'query': 'Denver'},
  'id': 'call_0emVUQf3inBMI0J81WU43KVu',
  'type': 'tool_call'}]

# Build and Test AI Agent
* We now have tools defined and the LLM setup, so we can create an agent.
* We will be using a tool calling agent to bind the tools to the agent with a prompt.
* We will also add in the capability to store historical conversations as memory.

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

## system prompt
SYS_PROMPT = """Act as a helpful assistant.
                Use the tools at your disposal to perform tasks as needed.
                  - get_weather: whenever user asks get the weather of a location.
                  - search_web: whenever user asks for information on current events or if you don't know the answer.
            """

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history", optional=True),
        ("human", "{query}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="Act as a helpful assistant.\n                Use the tools at your disposal to perform tasks as needed. \n                  - get_weather: whenever user asks get the weather of a location.\n                  - search_web: whenever user asks for information on current events or if you don't know the answer.\n            ")),
 MessagesPlaceholder(variable_name='history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

Summary so far:
* Now we can initialize the agent with the LLM, the prompt, and the tools.
* The agent is responsible for taking input and deciding what actions to take.
* REMEMBER the Agent does NOT execute those actions --- that is done by the `AgentExecutor`
* Note that we are passing in the model the LLM (`llm`) NOT the `llm_with_tools`.
  * That is because the `create_tool_calling_agent` will call `.bind_tools` for us under the hood.
  * This should ideally be used with an LLM which suppors tool/function calling.

In [15]:
from langchain.agents import create_tool_calling_agent

## agent
agent = create_tool_calling_agent(llm, tools, prompt_template)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'query'], optional_variables=['history'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'history': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="Act as a helpful assistant.\n 

Next, we combine the `agent` (the brains) with the `tools` inside the `AgentExecutor` which will repeatedly call the agent and execute tools.

In [16]:
from langchain.agents import AgentExecutor

## agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor

AgentExecutor(agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'query'], optional_variables=['history'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'history': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_

In [22]:
## now send query to agent
query = """Tell me who won the Stanley Cup in 2024,
        show me some detailed information about the game.
        """
response = llm.invoke(query)
response.content

"I'm sorry, but I can't provide real-time information or future events as my knowledge was last updated in October 2021. For the most current information on the Stanley Cup winner in 2024 and detailed information about the game, I recommend checking the latest sports news websites, the official NHL website, or other reliable sources."

We can see that the model does not know anything past a certain date so we can now use our agent to answer the query.

In [23]:
## now send query to agent
query = """Tell me who won the Stanley Cup in 2024,
        show me some detailed information about the game.
        """
response = agent_executor.invoke({"query": query})

In [24]:
## get response from agent executor
response

{'query': 'Tell me who won the Stanley Cup in 2024,\n        show me some detailed information about the game.\n        ',
 'output': "The Florida Panthers won the 2024 Stanley Cup, defeating the Edmonton Oilers in a thrilling seven-game series. This victory marked the Panthers' first championship in their thirty-year history. \n\nIn the decisive Game 7, the Panthers edged out the Oilers with a 2-1 victory. This win was a culmination of the 2023-24 NHL season and the 2024 Stanley Cup playoffs. \n\nFor more detailed information, you can check the [Wikipedia page](https://en.wikipedia.org/wiki/2024_Stanley_Cup_Finals) or the [Sporting News article](https://www.sportingnews.com/us/nhl/news/oilers-panthers-live-score-results-game-7-stanley-cup-final/73fb8b4b6c2b7816e0199225)."}

In [25]:
## cleanup the output with markdown
from IPython.display import display, Markdown

display(Markdown(response['output']))

The Florida Panthers won the 2024 Stanley Cup, defeating the Edmonton Oilers in a thrilling seven-game series. This victory marked the Panthers' first championship in their thirty-year history. 

In the decisive Game 7, the Panthers edged out the Oilers with a 2-1 victory. This win was a culmination of the 2023-24 NHL season and the 2024 Stanley Cup playoffs. 

For more detailed information, you can check the [Wikipedia page](https://en.wikipedia.org/wiki/2024_Stanley_Cup_Finals) or the [Sporting News article](https://www.sportingnews.com/us/nhl/news/oilers-panthers-live-score-results-game-7-stanley-cup-final/73fb8b4b6c2b7816e0199225).

In [26]:
## now lets try a weather query
query = """What is the weather in Denver, CO today?
        """

response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

The current weather in Denver, CO is partly cloudy with a temperature of 29.7°C (85.5°F). The wind is coming from the west-northwest at 2.2 mph (3.6 kph). The humidity is quite low at 6%, and there is no precipitation. The UV index is 8, indicating a high level of UV radiation, so take precautions if you're spending time outdoors. Visibility is good at 16 km (9 miles).

In [27]:
## how about another weather query
query = """What is the weather today in Stowe, VT?
        """

response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

The weather in Stowe, VT today is partly cloudy. The current temperature is 18.9°C (66.0°F). The wind is blowing from the south at 6.0 mph (9.7 kph). The humidity is at 54%, and visibility is 16 km (9 miles). The UV index is 3. 

![Partly cloudy](//cdn.weatherapi.com/weather/64x64/day/116.png)

In [28]:
## continue the conversation
query = """Which city is colder?"""

response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Could you please specify the cities you would like to compare in terms of temperature?

Summary:
* AHA! We have not used memory yet to store this.

# Build and Test Multi-User Conversational AI Agent
* We will now implement the `SQLChatMessageHistory` to store separate conversation histories per user or session.
* This will help us build a conversational Agentic Chatbot which will be accessed by many users at the same time.

In [29]:
## removes memory database file -- usually not needed
# you can run this only when you want to remove ALL conversation histories
## ok if you get rm: cannot remove 'memory.db`: No such file or directory because initially no memory exists
!rm memory.db

rm: cannot remove 'memory.db': No such file or directory


In [30]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


## used to retrieve conversation history from database
## based on specific user or session ID
def get_session_history_db(session_id):
  """Takes in session history and returns chat history from sqlite db"""
  return SQLChatMessageHistory(session_id, "sqlite:///memory.db")


## creates converation chain + agent which can load memory based on specific user or session id
agentic_chatbot = RunnableWithMessageHistory(
    agent_executor,
    get_session_history_db,
    input_messages_key='query', ##input prompt
    history_messages_key='history' ##history stored here
)

## function to call agent show results per user session
from IPython.display import display, Markdown
def chat_with_agent(prompt: str, session_id: str):
  response = agentic_chatbot.invoke({"query": prompt},
                                    {'configurable': { 'session_id': session_id}})
  display(Markdown(response['output']))

## Simulate User 1 using the agent

In [31]:
user_id = 'tom001'
prompt = 'Hello can you tell me who won the Superbowl in 2024'
chat_with_agent(prompt, user_id)

The Kansas City Chiefs won the Super Bowl in 2024, defeating the San Francisco 49ers 25-22 in overtime. This victory marked the Chiefs' third Super Bowl win in five years and their second consecutive title.

In [32]:
## continuing conversation
prompt = "Tell me more about this event in detail please"
chat_with_agent(prompt, user_id)

The Kansas City Chiefs won the Super Bowl in 2024, defeating the San Francisco 49ers 25-22 in an exciting overtime game. This victory marked the Chiefs' third Super Bowl win in five years and their second consecutive title. The game was held at Allegiant Stadium in Las Vegas on February 11, 2024.

Key highlights of the game include:

- The Chiefs' star quarterback, Patrick Mahomes, played a crucial role in the victory, throwing the game-winning touchdown pass to wide receiver Mecole Hardman Jr. during overtime.
- The game was only the second overtime match in Super Bowl history.
- The Chiefs' defense, led by a signature blitz from Steve Spagnuolo, forced a critical field goal from the 49ers late in the fourth quarter.
- San Francisco 49ers' running back Christian McCaffrey scored a touchdown during the first half, showcasing the 49ers' strong offensive capabilities.
- The Chiefs embraced an underdog mentality throughout the postseason, upsetting three teams to secure their second consecutive Super Bowl title.

The Chiefs' victory was celebrated with a steely resolve, as they arrived at the stadium in all black, signaling their determination to win against the NFC's best offensive team.

## Simulate User 2 using the agent

In [33]:
user_id = "taylor1989"
prompt = "how is the weather in Denver, CO today? Please show detailed stats"
chat_with_agent(prompt, user_id)

The weather in Denver, CO today is sunny. Here are the detailed statistics:

- **Temperature**: 31.4°C (88.5°F)
- **Feels Like**: 29.2°C (84.5°F)
- **Wind**: 6.5 mph (10.4 kph) from the northwest (NW)
- **Wind Gusts**: Up to 7.5 mph (12.0 kph)
- **Humidity**: 10%
- **Pressure**: 1017.0 mb (30.03 in)
- **Precipitation**: 0.0 mm (0.0 in)
- **Cloud Cover**: 0% (Clear skies)
- **Visibility**: 48.0 km (29.0 miles)
- **UV Index**: 8.0 (Very High)
- **Dew Point**: -3.0°C (26.6°F)

![Weather Icon](//cdn.weatherapi.com/weather/64x64/day/113.png)

It's a clear and sunny day with very low humidity, so make sure to stay hydrated and protect yourself from the sun if you're spending time outdoors!

In [34]:
## continue conversation
user_id = "taylor1989"
prompt = "What about Vail?"
chat_with_agent(prompt, user_id)

The weather in Vail, CO today is sunny. Here are the detailed statistics:

- **Temperature**: 15.1°C (59.2°F)
- **Feels Like**: 15.1°C (59.2°F)
- **Wind**: 11.2 mph (18.0 kph) from the west-northwest (WNW)
- **Wind Gusts**: Up to 12.9 mph (20.7 kph)
- **Humidity**: 14%
- **Pressure**: 1035.0 mb (30.56 in)
- **Precipitation**: 0.0 mm (0.0 in)
- **Cloud Cover**: 0% (Clear skies)
- **Visibility**: 16.0 km (9.0 miles)
- **UV Index**: 5.0 (Moderate)
- **Dew Point**: -4.6°C (23.7°F)

![Weather Icon](//cdn.weatherapi.com/weather/64x64/day/113.png)

It's a clear and sunny day in Vail with low humidity. Enjoy the beautiful weather!

In [35]:
## continue conversation
user_id = "taylor1989"
prompt = "which city is warmer?"
chat_with_agent(prompt, user_id)

Denver, CO is warmer than Vail, CO today. 

- **Denver**: 31.4°C (88.5°F)
- **Vail**: 15.1°C (59.2°F)

Denver's temperature is significantly higher than Vail's.